In [ ]:
# Imports
#!pip install nltk
#!pip install joblib
#!pip install gensim
#!pip install stop_words
##!pip install textblob
#!pip install xgboost
#!pip install lightgbm
import io, requests, operator, os
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import gensim
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
import math
from sklearn.metrics import mean_squared_error as mse
from gensim import corpora, models
from nltk.stem.snowball import SnowballStemmer
from gensim.models.phrases import Phraser
from collections import defaultdict
from stop_words import get_stop_words
from lib_aunsight.context import AunsightContext
import re
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from textblob import TextBlob


In [ ]:
pd.set_option('display.max_colwidth', -1)

### Load Data

In [ ]:
df = pd.read_csv('training_data_hackathon_clean.tsv',sep="\t")

### Feature Engineering

In [ ]:
df["sentiment"] = df['comment'].apply(lambda x: TextBlob(x).sentiment.polarity)
df["sentiment_sub"] = df['comment'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df["commentLength"] = df["comment"].str.len()
df["category"] = df["category"].astype('category')
df["category_code"] = df["category"].cat.codes

## Cleaning

In [ ]:
for i in training_cols:
    df[i].replace('None', np.nan, inplace=True)

for i in training_cols:
    df[i].fillna(df[i].median(), inplace=True)

In [ ]:
training_cols=[
#               'survey_number', 
               'age_cat', 
               'sex', 
               'stay_cat', 
               'lang', 
               'er',
               'category_code', 
               'sentiment', 
               'commentLength',
               'sentiment_sub'
              ]

In [ ]:
X = df[training_cols]

In [ ]:
y = df['score']

In [ ]:
X=X.astype(np.float, copy=True, errors='raise')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.95, random_state=42)

In [ ]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
'''
rf = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
'''

In [ ]:
rf = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=3000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1, n_jobs=-1)


In [ ]:
#rf = RandomForestRegressor(n_estimators=300, oob_score=True, random_state=0, n_jobs=-1, max_features='sqrt')


In [ ]:
pipeline = Pipeline([
#    ('imputer', preprocessing.Imputer(missing_values=np.nan, strategy='median')),
    ('scaler', preprocessing.StandardScaler()),
    ('rf', rf)
])
pipeline.fit(X_train, y_train)
# Get metrics on the hold out test set for validation
y_prediction = pipeline.predict(X_test)
y_prediction_rounded = np.round(y_prediction)
mean_sq_err = mse(y_test, y_prediction_rounded)
print('Confusion Matrix:\n', metrics.confusion_matrix(y_test, y_prediction_rounded))
print('Classification report:\n', metrics.classification_report(y_test, y_prediction_rounded))
print(f'Test data MSE : {mean_sq_err:.10}')

## Scoring

In [ ]:
test = pd.read_csv('test_data.csv',sep="\t")

In [ ]:
test.shape

In [ ]:
test.columns

In [ ]:
test["sentiment"] = test['comment'].apply(lambda x: TextBlob(x).sentiment.polarity)
test["sentiment_sub"] = test['comment'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
test["commentLength"] = test["comment"].str.len()

In [ ]:
test.age_cat.value_counts()

In [ ]:
test["category"] = test["category"].astype('category')
test["category_code"] = test["category"].cat.codes

In [ ]:
for i in training_cols:
    test[i].replace('None', np.nan, inplace=True)

In [ ]:
for i in training_cols:
    test[i].fillna(test[i].median(), inplace=True)

In [ ]:
test_data = test[training_cols]

In [ ]:
y_prediction_test = pipeline.predict(test_data)

In [ ]:
test.predictions =test.predictions.astype(np.int, copy=True, errors='raise')

In [ ]:
df.score.value_counts(dropna=False,normalize=True)

In [ ]:
test.predictions.value_counts(dropna=False,normalize=True)

In [ ]:
test.to_csv('test_scored.tsv',sep="\t", index=False)